


# Assignment 5 on Natural Language Processing

## Date : 3rd Nov, 2020

### Instructor : Prof. Sudeshna Sarkar

### Teaching Assistants : Alapan Kuila, Aniruddha Roy, Anusha Potnuru, Uppada Vishnu

The central idea of this assignment is to explore various language models specifically LSTM based and transformer. We will explore how the size of the model effects the sequence generated. We will see both character based and word based models.



Please submit with outputs. Submissions without predicted outputs will be penalized.

# Word Based LSTM model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras

Do basic pre processing which includes lowering etc
Check the dataset and apply suitable preprocessing.

In [ ]:
# Load the data and preprocess data and store corpus in raw_text
import io
path = "./corpus.txt"
def preprocess(text=None, path=None):
  if path != None:
    with io.open(path, encoding="utf-8") as f:
        text = f.read().lower()
  else:
    text = text.lower()

  # Removing unnecessary characters
  text = text.replace("\n", " ")
  text = text.replace("_", "")
  text = text.replace("—", " ")
  text = text.replace("-", " ")
  text = text.replace("(", "")
  text = text.replace(")", "")
  text = text.replace("[", "")
  text = text.replace("]", "")
  text = text.replace("  ", " ") # Above replacements may cause double spaces, this is to fix that

  char_level = text
#  word_level = re.sub('[\W_]+', '', text)
  word_level = re.sub('[^a-z\d\s]', '', char_level) # Removing all special characters for word level modelling
  return char_level, word_level

_,raw_text = preprocess(path=path)

In [ ]:
# Hyperparameters of the model
vocab_size = 2613 # choose based on statistics
oov_tok = '<OOV>'
embedding_dim = 100
padding_type='post'
trunc_type='post'

In [ ]:
# tokenize sentences
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts([raw_text])
word_index = tokenizer.word_index

In [ ]:
seq_length = 50
tokens = tokenizer.texts_to_sequences([raw_text])[0]

In [ ]:
dataX = []
dataY = []

for i in range(0, len(tokens) - seq_length-1 , 1):
  seq_in = tokens[i:i + seq_length]
  seq_out = tokens[i + seq_length]

  if seq_out==1: #Skip samples where target word is OOV
    continue
    
  dataX.append(seq_in)
  dataY.append(seq_out)
 
N = len(dataX)
print ("Total training data size: ", N)

Total training data size:  26635


In [ ]:
X = numpy.array(dataX).reshape(len(dataX), len(dataX[0]), 1)
# one hot encode the output variable
y = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [ ]:
# with embedding
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=seq_length),
    keras.layers.Bidirectional(keras.layers.LSTM(256)),
    keras.layers.Dense(vocab_size, activation='softmax')
])
# compile model
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# model summary
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 50, 100)           261300    
_________________________________________________________________
bidirectional (Bidirectional (None, 512)               731136    
_________________________________________________________________
dense (Dense)                (None, 2613)              1340469   
Total params: 2,332,905
Trainable params: 2,332,905
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Use validation split of 0.2 while training
model.fit(X, y, epochs=25, batch_size=32, verbose=1, validation_split=0.2) 

Epoch 1/25
666/666 [==============================] - 11s 16ms/step - loss: 6.1616 - accuracy: 0.0590 - val_loss: 6.1098 - val_accuracy: 0.0897
Epoch 2/25
666/666 [==============================] - 10s 15ms/step - loss: 5.6493 - accuracy: 0.0737 - val_loss: 6.0035 - val_accuracy: 0.1059
Epoch 3/25
666/666 [==============================] - 10s 15ms/step - loss: 5.2299 - accuracy: 0.1016 - val_loss: 5.9452 - val_accuracy: 0.1158
Epoch 4/25
666/666 [==============================] - 10s 15ms/step - loss: 4.7256 - accuracy: 0.1328 - val_loss: 6.0125 - val_accuracy: 0.1149
Epoch 5/25
666/666 [==============================] - 10s 15ms/step - loss: 4.1336 - accuracy: 0.1737 - val_loss: 6.1914 - val_accuracy: 0.1119
Epoch 6/25
666/666 [==============================] - 10s 15ms/step - loss: 3.4290 - accuracy: 0.2561 - val_loss: 6.4662 - val_accuracy: 0.0982
Epoch 7/25
666/666 [==============================] - 10s 15ms/step - loss: 2.7121 - accuracy: 0.3929 - val_loss: 6.7801 - val_accuracy:

In [ ]:
reverse_word_map = {value: key for key, value in tokenizer.word_index.items()}

In [ ]:
# Complete the code to return next n words greedily
def next_tokens(input_str, n): 
	print ("Seed: \n", input_str)
	input_string = input_str
	final_string = ""
	for i in range(n):
		final_string += " "
		new_X = np.asarray(tokenizer.texts_to_sequences([input_string])[0]).reshape(1, 50, 1)
		prediction = model.predict(new_X, verbose=0)
		# get next word index. Use reverse_word_map to get the word
		pred_word = reverse_word_map[np.argmax(prediction)]
		input_string = input_string + " " + pred_word
		final_string += pred_word
		input_string = ' '.join(input_string.split(" ")[1:])
	return final_string

In [ ]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ' '.join([reverse_word_map[value] for value in pattern])

print(next_tokens(input_str, 25))

Seed: 
 twice half hoping that they would call after her the last time she saw them they were trying to put the dormouse into the teapot at any rate ill never go there again said alice as she picked her way through the wood its the stupidest tea party i ever
 was at in all my life just as she said this she noticed that one of the trees had a door leading right into it


In [ ]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

# Use first 50 tokens from given input_str as input.(Use tokenizer to split to take first 50)
_,prep_input = preprocess(text=input_str)
input = ' '.join(prep_input.split(" ")[:50])
print(next_tokens(input , 10))

Seed: 
 the boy laughed at the fright he had caused this time the villagers left angrily the third day as the boy went up the small hill he suddenly saw a wolf attacking his sheep he cried as hard as he could wolf wolf wolf but not a single villager came
 in the executioner thought it to her with a game


# Character based LSTM Model 1

In [ ]:
# User the preprocess data and create raw_text
raw_text,_ = preprocess(path=path)
# create mapping of unique characters to integers
chars = sorted(list(set(raw_text)))

char_to_int = {c: i for i, c in enumerate(chars)}

In [ ]:
# Print the total characters and character vacob size
n_chars = len(raw_text)
n_vocab = len(chars)

print(n_chars, n_vocab)

141898 38


In [ ]:
'''
Prepare dataset where the input is sequence of 100 characters and target is next character.
'''
char_nums = [char_to_int[char] for char in raw_text]
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
  seq_in = char_nums[i:i + seq_length]
  seq_out = char_nums[i + seq_length]

  dataX.append(seq_in)
  dataY.append(seq_out)

n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)

Total Patterns:  141798


In [ ]:
# reshape X to be [samples, time steps, features]
X = numpy.array(dataX).reshape(len(dataX), len(dataX[0]), 1)
# one hot encode the output variable
dataY = numpy.array(dataY)
y = np_utils.to_categorical(dataY)

In [ ]:
embedding_dim = 100
max_length = 100

In [ ]:
model = Sequential()
model.add(keras.layers.Embedding(n_vocab, embedding_dim, input_length=max_length))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          3800      
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               365568    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 38)                9766      
Total params: 379,134
Trainable params: 379,134
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, y, epochs=25, batch_size=64)

Epoch 1/25
2216/2216 [==============================] - 28s 13ms/step - loss: 2.0273
Epoch 2/25
2216/2216 [==============================] - 28s 12ms/step - loss: 1.5616
Epoch 3/25
2216/2216 [==============================] - 28s 12ms/step - loss: 1.3967
Epoch 4/25
2216/2216 [==============================] - 28s 12ms/step - loss: 1.2994
Epoch 5/25
2216/2216 [==============================] - 28s 12ms/step - loss: 1.2305
Epoch 6/25
2216/2216 [==============================] - 28s 12ms/step - loss: 1.1738
Epoch 7/25
2216/2216 [==============================] - 28s 12ms/step - loss: 1.1295
Epoch 8/25
2216/2216 [==============================] - 28s 12ms/step - loss: 1.0928
Epoch 9/25
2216/2216 [==============================] - 28s 12ms/step - loss: 1.0574
Epoch 10/25
2216/2216 [==============================] - 28s 12ms/step - loss: 1.0272
Epoch 11/25
2216/2216 [==============================] - 28s 12ms/step - loss: 1.0004
Epoch 12/25
2216/2216 [==============================] - 28s 13

In [ ]:
#implement mapping of integer to character
int_to_char = {i: c for i, c in enumerate(chars)}

In [ ]:
'''
Complete code below to get the generated string using the model.
'''
def predict_next_100_chars(pattern,x):
	input_pattern = pattern
	final_string = ""
	for i in range(x):
		new_X = np.array([(char_to_int[char]) for char in input_pattern])
		new_X = new_X.reshape((1, len(pattern), 1))
		prediction = model.predict(new_X, verbose=0)
#		pred_sampled = np.random.choice(len(prediction[0]), p=prediction[0])
		pred_sampled = np.argmax(prediction[0])
		pred_char = int_to_char[pred_sampled]
		final_string += pred_char
		input_pattern += pred_char
		input_pattern = input_pattern[1:]
	return final_string

In [ ]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])
print("Seed:\n", input_str)

print(predict_next_100_chars(input_str,200))

Seed:
 , “i feared it might injure the brain; but, now that i’m perfectly sure i have none, why, i do it ag
ain the door of the end of the world ear was some of the english,” the gryphon asked the mock turtle as she went on, “you were never marked it was a little way of meaning in a treecle, for the things 


In [ ]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.
input_str = input_str[:100].lower()
print("Seed: ", input_str)
print(predict_next_100_chars(input_str,200))

Seed:  the boy laughed at the fright he had caused. this time, the villagers left angrily. the third day, a
nd she was silence to herself herself to alice who had to such a thing it as she was so much at the court, and then alice could not stand on the table, but then she was silently that she was quite ple


## Character based LSTM Model 2


In [ ]:
model1 = Sequential()
model1.add(keras.layers.Embedding(n_vocab, embedding_dim, input_length=max_length))
model1.add(LSTM(256, input_shape=(X.shape[1], embedding_dim),return_sequences=True))
model1.add(Dropout(0.2))
model1.add(LSTM(256))
model1.add(Dropout(0.2))
model1.add(Dense(y.shape[1], activation='softmax'))
model1.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
model1.fit(X, y, epochs=20, batch_size=64)

Epoch 1/20
2216/2216 [==============================] - 52s 23ms/step - loss: 2.0568
Epoch 2/20
2216/2216 [==============================] - 52s 23ms/step - loss: 1.5370
Epoch 3/20
2216/2216 [==============================] - 52s 23ms/step - loss: 1.3622
Epoch 4/20
2216/2216 [==============================] - 52s 23ms/step - loss: 1.2661
Epoch 5/20
2216/2216 [==============================] - 52s 23ms/step - loss: 1.1975
Epoch 6/20
2216/2216 [==============================] - 52s 23ms/step - loss: 1.1454
Epoch 7/20
2216/2216 [==============================] - 52s 23ms/step - loss: 1.1008
Epoch 8/20
2216/2216 [==============================] - 52s 23ms/step - loss: 1.0642
Epoch 9/20
2216/2216 [==============================] - 52s 23ms/step - loss: 1.0339
Epoch 10/20
2216/2216 [==============================] - 52s 23ms/step - loss: 1.0037
Epoch 11/20
2216/2216 [==============================] - 52s 23ms/step - loss: 0.9814
Epoch 12/20
2216/2216 [==============================] - 52s 23

In [ ]:
# Generate the sequence similar to above methods

'''
Complete code below to get the generated string using the model.
'''

def predict_next_100_chars(pattern,x):
	input_pattern = pattern
	final_string = ""
	for i in range(x):
		new_X = np.array([(char_to_int[char]) for char in input_pattern])
		new_X = new_X.reshape((1, len(pattern), 1))
		prediction = model1.predict(new_X, verbose=0)
#		pred_sampled = np.random.choice(len(prediction[0]), p=prediction[0])
		pred_sampled = np.argmax(prediction[0])
		pred_char = int_to_char[pred_sampled]
		final_string += pred_char
		input_pattern += pred_char
		input_pattern = input_pattern[1:]
	return final_string

In [ ]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
input_str = ''.join([int_to_char[value] for value in pattern])

print(predict_next_100_chars(input_str,200))

 words were three of the words “drink me the rest of the sea. “i don’t know what they were orderand, i wonder what i say that is to grow about it, you know, with the dormouse,” said the king. “nothing


In [ ]:
input_str = "The boy laughed at the fright he had caused. This time, the villagers left angrily. The third day, as the boy went up\
 the small hill, he suddenly saw a wolf attacking his sheep. He cried as hard as he could, “Wolf! Wolf! Wolf!”, but not \
 a single villager came to help him. The villagers thought that he was trying to fool them again and did not come to rescue \
 him or his sheep."

 # Use first 100 characeters from given input_str as input and generate next 200 characters.
input_str = input_str[:100].lower()
 
print(predict_next_100_chars(input_str,200))

nd she went on, “i wonder what i say that is to grow about it, you know, with the dormouse,” said the king. “nothing would not, could not, could not, could not, could not, could not, could not, could 


# Questions

**Question:** What are your observations based on the model(all) outputs on train data(in domain) vs unseen data(out of domain) ?

**Answer:** The correlation between seed text and generated text is much more on the train data than on the unseen data. This is most starkly observed in the word level model.
The characters from the input text such as alice, caterpillar, hatter, etc show up in the generated text regardless of the seed.

**Question:** What did you observe in the outputs of char LSTM model1 vs char LSTM model2 ?

**Answer:** Seen over multiple text generations and experimentation with longer predictions as well, the following things are observed:
* Due to the second LSTM layer, the second model captures deeper representations and longer range dependencies better than the first model.
* Both models start looping when sampling is done greedily (as opposed to the more optimal Beam Search) but the second model has longer loops, indicating a deeper understanding of the corpus it is trained on.

# Bonus (Not to be graded)

## Transformer based language model (Bert)
You can run the below cells to predict the next word using pretrained BERT model. 

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.3MB 5.9MB/s 
     |████████████████████████████████| 1.1MB 28.5MB/s 
     |████████████████████████████████| 2.9MB 42.9MB/s 
     |████████████████████████████████| 890kB 47.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=95301894fcbcc09b8dad43504e9729ca76001a21e8360893555ad3258c86a23f
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import os
import torch
import string
from transformers import BertTokenizer, BertForMaskedLM

In [ ]:
def load_model(model_name):
  try:
    if model_name.lower() == "bert":
      bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
      bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased').eval()
      return bert_tokenizer,bert_model
  except Exception as e:
    pass

In [ ]:
def decode(tokenizer, pred_idx, top_clean):
  ignore_tokens = string.punctuation + '[PAD]'
  tokens = []
  for w in pred_idx:
    token = ''.join(tokenizer.decode(w).split())
    if token not in ignore_tokens:
      tokens.append(token.replace('##', ''))
  return '\n'.join(tokens[:top_clean])

In [ ]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
  text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
  # if <mask> is the last token, append a "." so that models dont predict punctuation.
  if tokenizer.mask_token == text_sentence.split()[-1]:
    text_sentence += ' .'
  input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
  mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
  return input_ids, mask_idx

In [ ]:
def get_all_predictions(text_sentence, top_clean=5):
  input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
  with torch.no_grad():
    predict = bert_model(input_ids)[0]
    print(predict.shape)
    
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)
  return {'bert': bert}

In [ ]:
def get_prediction_eos(input_text):
  try:
    input_text += ' <mask>'
    res = get_all_predictions(input_text, top_clean=int(top_k))
    return res
  except Exception as error:
    pass

In [ ]:
# Below code predicts the next top_k words. You can modify this to get next n words using top_k=1 and greedy decoding it. 
top_k= 3
print('Predict next ', top_k, ' words')
model_name = 'BERT'
bert_tokenizer, bert_model  = load_model(model_name) 
input_text = "The boy" ### GIVE YOUR INPUT STRING HERE
res = get_prediction_eos(input_text)
answer = []
print(res['bert'].split("\n"))
for i in res['bert'].split("\n"):
  answer.append(i)
  answer_as_string = "    ".join(answer)

print(answer_as_string)

Predict next  3  words


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([1, 6, 30522])
['asked', 'nodded', 'said']
asked    nodded    said
